In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [2]:
# Define a function for Mean Squared Error (MSE)
def fMSE(Y1, Y2):
    return np.mean((Y1 - Y2) ** 2)

In [3]:
# Read the data
data = pd.read_csv("stores.csv")
nn = len(data)

# Transform the outcome variable
data['Rev2'] = data['revenue'] ** 2
data['income_x_competition'] = data['income'] * data['competition']
data = data.drop(columns=['revenue'])  # Remove the original revenue variable

In [4]:
# List all the models you want to try
all_models = [
    ['income'],
    ['competition'],
    ['income_x_competition'],
    ['income', 'competition'],
    ['income', 'income_x_competition'],
    ['competition', 'income_x_competition'],
    ['income', 'competition', 'income_x_competition']
]

In [5]:
######################
# In-sample approach
######################
for mm in range(7):
    X_train = data[all_models[mm]]
    X_train = sm.add_constant(X_train)  # Add a constant for the intercept
    model_train = sm.OLS(data['Rev2'], X_train).fit()
    predictions = model_train.predict()
    MSE = fMSE(data['Rev2'], predictions)
    print(f'MSE = {MSE} for model {all_models[mm]}')

MSE = 29317869607.4817 for model ['income']
MSE = 390452510068.7267 for model ['competition']
MSE = 357115573971.6504 for model ['income_x_competition']
MSE = 15222770162.002348 for model ['income', 'competition']
MSE = 6297100897.987369 for model ['income', 'income_x_competition']
MSE = 265226137959.80945 for model ['competition', 'income_x_competition']
MSE = 6288823037.595658 for model ['income', 'competition', 'income_x_competition']


In [6]:
# Choose the final model
X_train = data[['income', 'competition', 'income_x_competition']]
X_train = sm.add_constant(X_train)  # Add a constant for the intercept
final_model = sm.OLS(data['Rev2'], X_train).fit()

In [7]:
# Predict a new observation
newdata = pd.DataFrame({'const':[1], 'income': [350], 'competition': [1], 'income_x_competition': [350]})
predictions = final_model.get_prediction(newdata)
print(predictions.summary_frame())

           mean      mean_se  mean_ci_lower  mean_ci_upper  obs_ci_lower  \
0  1.467945e+06  5022.152326   1.458089e+06   1.477801e+06  1.311677e+06   

   obs_ci_upper  
0  1.624213e+06  


In [8]:
## Prediction in the original scale
my_predictions =  pd.DataFrame({'predicted_revenue': np.sqrt(predictions.summary_frame()['mean'])})
my_predictions.head()

,predicted_revenue
0,1211.587676


In [9]:

###########################
# Train/validate approach
############################
train_size = int(0.7 * nn)
validate_size = nn - train_size
random_train_rows = np.random.choice(nn, train_size, replace=False)
random_validate_rows = np.setdiff1d(np.arange(nn), random_train_rows)
train_data = data.iloc[random_train_rows]
validate_data = data.iloc[random_validate_rows]
validate_data = sm.add_constant(validate_data)

In [11]:
random_train_rows

array([185, 288, 797, 460, 265, 738, 144,  62, 137, 414,  43, 429, 169,
       912, 470, 427,  24, 874, 332, 473, 601, 799, 867, 769, 730, 389,
       747, 513,  82, 795, 452, 505, 651, 465, 922, 717, 897, 377, 106,
       161, 847, 616, 900, 868, 896, 657, 247, 729, 939,   2, 681, 126,
       599, 549, 393, 945,  42, 839, 654,  56,  78, 905, 319, 384, 209,
       735,  27, 248, 936, 714, 323, 146, 533, 534, 798, 942, 558, 371,
       724, 170, 563, 630,  87, 487, 446, 906, 817, 784, 346, 665, 895,
       255, 556, 167, 864, 272, 491, 609, 147, 340, 869, 819,  50, 700,
       914, 788, 348, 668, 760, 285, 583, 325, 578,  96, 585, 767, 755,
       933, 176, 103, 229, 481, 338, 870, 301, 205, 615, 584, 759, 559,
       334, 749, 916, 943, 364, 391, 701, 540, 688,  23,   3, 623, 946,
       469, 511, 308, 309, 228, 631, 485, 501, 191, 111, 454, 136, 841,
       215, 545, 814,  77, 529, 638, 804, 149, 836, 852, 596, 503, 117,
        61,  41, 447, 838, 274, 425, 586,  16, 748,  75,  79, 27

,const,income,competition,Rev2,income_x_competition
0,1.0,690.977163,0,3.367561e+06,0.0
1,1.0,160.078168,0,1.058376e+06,0.0
4,1.0,372.866534,0,1.909168e+06,0.0
7,1.0,301.879729,0,1.767682e+06,0.0
13,1.0,124.455997,0,9.609142e+05,0.0


In [52]:
for mm in range(7):
    X_train = train_data[all_models[mm]]
    X_train = sm.add_constant(X_train)  # Add a constant for the intercept
    model_train = sm.OLS(train_data['Rev2'], X_train).fit()
    X_validate = validate_data[all_models[mm]]
    X_validate = sm.add_constant(X_validate)
    predictions_on_validation = model_train.predict(X_validate)
    MSE = fMSE(validate_data['Rev2'], predictions_on_validation)
    print(f'MSE = {MSE} for model {all_models[mm]}')

MSE = 27923249954.91975 for model ['income']
MSE = 354306138895.22925 for model ['competition']
MSE = 309039041145.0555 for model ['income_x_competition']
MSE = 15501099754.450521 for model ['income', 'competition']
MSE = 6258781135.952013 for model ['income', 'income_x_competition']
MSE = 234461199715.92642 for model ['competition', 'income_x_competition']
MSE = 6343863148.001071 for model ['income', 'competition', 'income_x_competition']


In [53]:
# Choose the final model
X_train = data[['income', 'income_x_competition']]
X_train = sm.add_constant(X_train)  # Add a constant for the intercept
final_model = sm.OLS(data['Rev2'], X_train).fit()

In [67]:
# Predict a new observation
newdata = pd.DataFrame({'const':[1], 'income': [350], 'income_x_competition': [350]})
predictions = final_model.get_prediction(newdata)
print(predictions.summary_frame())

           mean      mean_se  mean_ci_lower  mean_ci_upper  obs_ci_lower  \
0  1.467819e+06  5021.538864   1.457964e+06   1.477674e+06  1.311531e+06   

   obs_ci_upper  
0  1.624107e+06  


In [68]:
## Prediction in the original scale
my_predictions =  pd.DataFrame({'predicted_revenue': np.sqrt(predictions.summary_frame()['mean'])})
my_predictions.head()

,predicted_revenue
0,1211.535802


In [69]:
###############################
# Cross-validation approach
###############################
train_size = int(0.7 * nn)
validate_size = nn - train_size
SS = 100
all_MSE = np.empty((SS, 7))

for ss in range(SS):
    random_train_rows = np.random.choice(nn, train_size, replace=False)
    random_validate_rows = np.setdiff1d(np.arange(nn), random_train_rows)
    train_data = data.iloc[random_train_rows]
    validate_data = data.iloc[random_validate_rows]

    for mm in range(7):
        X_train = train_data[all_models[mm]]
        X_train = sm.add_constant(X_train)  # Add a constant for the intercept
        model_train = sm.OLS(train_data['Rev2'], X_train).fit()
        X_validate = validate_data[all_models[mm]]
        X_validate = sm.add_constant(X_validate)
        predictions_on_validation = model_train.predict(X_validate)
        all_MSE[ss, mm] = fMSE(validate_data['Rev2'], predictions_on_validation)

In [70]:
Average_MSEs = np.mean(all_MSE, axis=0)

for mm in range(7):
    print(f'Average_MSE = {Average_MSEs[mm]} for model {all_models[mm]}')

Average_MSE = 29706082484.999146 for model ['income']
Average_MSE = 392490427934.8388 for model ['competition']
Average_MSE = 360460735260.4269 for model ['income_x_competition']
Average_MSE = 15357222999.256916 for model ['income', 'competition']
Average_MSE = 6325227819.5311165 for model ['income', 'income_x_competition']
Average_MSE = 266843842404.86484 for model ['competition', 'income_x_competition']
Average_MSE = 6330969018.569842 for model ['income', 'competition', 'income_x_competition']


In [71]:
# Choose the final model
X_train = data[['income', 'income_x_competition']]
X_train = sm.add_constant(X_train)  # Add a constant for the intercept
final_model = sm.OLS(data['Rev2'], X_train).fit()

In [72]:
# Predict a new observation
newdata = pd.DataFrame({'const':[1], 'income': [350], 'income_x_competition': [350]})
predictions = final_model.get_prediction(newdata)
print(predictions.summary_frame())

           mean      mean_se  mean_ci_lower  mean_ci_upper  obs_ci_lower  \
0  1.467819e+06  5021.538864   1.457964e+06   1.477674e+06  1.311531e+06   

   obs_ci_upper  
0  1.624107e+06  


In [73]:
## Prediction in the original scale
my_predictions =  pd.DataFrame({'predicted_revenue': np.sqrt(predictions.summary_frame()['mean'])})
my_predictions.head()

,predicted_revenue
0,1211.535802
